In [1]:
cd ../../

/home/divye/Documents/Projects/mic_predictor


In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from notebooks.utils import *
import importlib
import sys
importlib.reload(sys.modules['notebooks.utils'])
from notebooks.utils import *

In [5]:
results = pd.read_csv("results/knn_regressor_all_data_combination_separate_prop_pIC50.csv", sep=';')

In [6]:
results

,Data,Regressor,Best_parameters,MSE,MAE,MAPE
0,aa_freq,KNN,"{'metric': 'manhattan', 'n_neighbors': 10, 'we...",6.404887,1.783988,15.334217
1,dipep_freq,KNN,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",4.929638,1.471659,12.267070
2,avg_protvec,KNN,"{'metric': 'euclidean', 'n_neighbors': 2, 'wei...",6.498897,1.783781,15.057070
3,shannon_entropy,KNN,"{'metric': 'manhattan', 'n_neighbors': 11, 'we...",5.897072,1.730841,14.705748
4,residue_repeats,KNN,"{'metric': 'manhattan', 'n_neighbors': 6, 'wei...",5.808975,1.667210,13.841934
...,...,...,...,...,...,...
2051,aa_freq+dipep_freq+avg_protvec+residue_repeats...,KNN,"{'metric': 'manhattan', 'n_neighbors': 6, 'wei...",4.838845,1.542169,12.804184
2052,aa_freq+dipep_freq+shannon_entropy+residue_rep...,KNN,"{'metric': 'manhattan', 'n_neighbors': 7, 'wei...",5.150265,1.595945,13.396485
2053,aa_freq+avg_protvec+shannon_entropy+residue_re...,KNN,"{'metric': 'manhattan', 'n_neighbors': 7, 'wei...",5.001320,1.563774,13.091918
2054,dipep_freq+avg_protvec+shannon_entropy+residue...,KNN,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",5.286135,1.587492,13.081379


In [8]:
print(f"Best MAPE: {results['MAPE'].min()}")
print(f"Best performing data: {results.loc[results['MAPE'].argmin()]['Data']}")
print(f"Best parameters: {results.loc[results['MAPE'].argmin()]['Best_parameters']}")

Best MAPE: 11.93221791296319
Best performing data: dipep_freq+turn
Best parameters: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}


### Analysis

In [ ]:
dipep_freq = reduce_by_kmer_frequency(df, kmer=2)

In [ ]:
uniprot_embedding = biovec.models.load_protvec("data/embeddings/uniprot__kmer_3_contextWindow_10_vector_100_reduction_None")

avg_protvec = convert_sequences_to_avg_vectors(df['Sequence'], uniprot_embedding)
avg_protvec = avg_protvec.reset_index(drop=True)

In [ ]:
dipep_protvec = pd.concat([avg_protvec, dipep_freq], axis=1)

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
def plot_nbr_mic(data, target, n_neighbors=1, metric='euclidean', n_jobs=10):
    data = data.reset_index(drop=True)
    target = target.reset_index(drop=True)
    nearesr_neighbors = NearestNeighbors(n_neighbors=n_neighbors, metric=metric, n_jobs=n_jobs)
    nearesr_neighbors.fit(data)
    grph = nearesr_neighbors.kneighbors_graph(n_neighbors=n_neighbors).todense()
    neighbors = np.where(grph == 1)
    neighbors = np.array([[i]+neighbors[1][np.where(neighbors[0]==i)[0]].tolist() for i in range(data.shape[0])])
    mic_nbrs = np.full(neighbors.shape, np.nan)
    for i in range(mic_nbrs.shape[1]):
        mic_nbrs[:,i] = target[neighbors[:,i]]

    plt.figure(figsize=(5*mic_nbrs.shape[1],50))
    im = plt.imshow(mic_nbrs, cmap='gist_ncar', interpolation='none') #plt.cm.get_cmap('gist_rainbow', 75)
    cbar = plt.colorbar(im)
    cbar.ax.tick_params(labelsize=20)
    cbar.set_label('MIC', fontsize=25)
    plt.axis('auto')
    plt.xlabel('neighbors', fontdict={'size':5*mic_nbrs.shape[1]})
    plt.ylabel('#row', fontdict={'size':5*mic_nbrs.shape[1]})
    plt.xticks(fontsize=5*mic_nbrs.shape[1])
    plt.yticks(fontsize=5*mic_nbrs.shape[1])
    plt.show()

### Observation

For kNN to give good results, sequences which are closer together should have MIC values closer together. But it is seen from below plot that many of the sequences with higher MIC values have low MIC value sequeces as their neighbors. However, as we look at lower MIC value sequences, they start to have low MIC value sequeces as their neighbors.

In [ ]:
plot_nbr_mic(dipep_protvec, df['MIC'], n_neighbors=5)

In [ ]:
mask = df['MIC'] < 5

In [ ]:
plot_nbr_mic(dipep_protvec[mask], df['MIC'][mask], n_neighbors=1)